In [2]:
import pandas as pd
import numpy as np
raw = pd.read_csv("research data.csv",";")
raw

,country,eiu,year,ISO 3166 name,country code,default probability
0,Canada,9.24,2020,Canada,CA,6.508
1,Canada,9.22,2019,Canada,CA,6.574
2,Canada,9.15,2018,Canada,CA,6.513
3,Canada,9.15,2017,Canada,CA,6.471
4,Canada,9.15,2016,Canada,CA,6.398
...,...,...,...,...,...,...
2166,Zimbabwe,2.67,2012,Zimbabwe,ZW,Invalid RIC(s): ZWGV10YUSAC=R
2167,Zimbabwe,2.68,2011,Zimbabwe,ZW,Invalid RIC(s): ZWGV10YUSAC=R
2168,Zimbabwe,2.64,2010,Zimbabwe,ZW,Invalid RIC(s): ZWGV10YUSAC=R
2169,Zimbabwe,2.53,2008,Zimbabwe,ZW,Invalid RIC(s): ZWGV10YUSAC=R


In [3]:
df = raw.copy()
df["default probability"] = pd.to_numeric(df["default probability"],errors='coerce')/100
df = df.dropna()
df = df.drop(columns=["ISO 3166 name","country code"])
df["eiu"]/=10
df.columns = ["country","demT","year","probT"]
df = df[["country","year","demT","probT"]]
df

,country,year,demT,probT
0,Canada,2020,0.924,0.06508
1,Canada,2019,0.922,0.06574
2,Canada,2018,0.915,0.06513
3,Canada,2017,0.915,0.06471
4,Canada,2016,0.915,0.06398
...,...,...,...,...
2099,South Africa,2014,0.782,0.29183
2100,South Africa,2013,0.790,0.31647
2101,South Africa,2012,0.779,0.25572
2102,South Africa,2011,0.779,0.27705


In [4]:
#observations
df.count()["country"]

719

In [5]:
#countries
len(df["country"].unique())

77

In [32]:
#years
df.count()["country"]/len(df["country"].unique())

9.337662337662337

In [34]:
df[["year","demT","probT"]].min()

year     2010.000
demT        0.171
probT       0.000
dtype: float64

In [7]:
df[f"probT-{1}"] = df.groupby("country").shift(-1)["probT"]
df[f"probDelta{1}"] = df["probT"]-df[f"probT-{1}"]
for i in [1,2,3]:
    df[f"demT-{i}"] = df.groupby("country").shift(-i)["demT"]
    df[f"demDelta{i}"] = df["demT"]-df[f"demT-{i}"]
df

,country,year,demT,probT,probT-1,probDelta1,demT-1,demDelta1,demT-2,demDelta2,demT-3,demDelta3
0,Canada,2020,0.924,0.06508,0.06574,-0.00066,0.922,0.002,0.915,0.009,0.915,0.009
1,Canada,2019,0.922,0.06574,0.06513,0.00061,0.915,0.007,0.915,0.007,0.915,0.007
2,Canada,2018,0.915,0.06513,0.06471,0.00042,0.915,0.000,0.915,0.000,0.908,0.007
3,Canada,2017,0.915,0.06471,0.06398,0.00073,0.915,0.000,0.908,0.007,0.908,0.007
4,Canada,2016,0.915,0.06398,0.06679,-0.00281,0.908,0.007,0.908,0.007,0.908,0.007
...,...,...,...,...,...,...,...,...,...,...,...,...
2099,South Africa,2014,0.782,0.29183,0.31647,-0.02464,0.790,-0.008,0.779,0.003,0.779,0.003
2100,South Africa,2013,0.790,0.31647,0.25572,0.06075,0.779,0.011,0.779,0.011,0.779,0.011
2101,South Africa,2012,0.779,0.25572,0.27705,-0.02133,0.779,0.000,0.779,0.000,NaN,NaN
2102,South Africa,2011,0.779,0.27705,0.19880,0.07825,0.779,0.000,NaN,NaN,NaN,NaN


In [8]:
pd.DataFrame({column:{
    "mean":df[column].mean(),
    "median":df[column].median(),
    "std":df[column].std(),
    "min":df[column].min(),
    "max":df[column].max(),
} for column in ["demT","probT","demDelta1","probDelta1",]}).transpose()

,mean,median,std,min,max
demT,0.650542,0.685000,0.182216,0.17100,0.99300
probT,0.271494,0.222520,0.187012,0.00000,0.99410
demDelta1,-0.001760,0.000000,0.019147,-0.12900,0.16900
probDelta1,-0.002697,-0.005015,0.082507,-0.45251,0.47298


In [17]:
import plotly.figure_factory as ff
p = ff.create_distplot([df["demT"].dropna()],['dem_t'],bin_size=.01)
p.write_image("demT.png")
p

In [20]:
import plotly.figure_factory as ff
p = ff.create_distplot([df["demDelta1"].dropna()],['dem_t - dem_{t-1}'],bin_size=.005)
p.write_image("demDelta1.png")
p

In [24]:
p = ff.create_distplot([df["probT"].dropna()],['prob_t'],bin_size=.01)
p.write_image("probT.png")
p

In [25]:
p = ff.create_distplot([df["probDelta1"].dropna()],['prob_t - prob_{t-1}'],bin_size=.01)
p.write_image("probDelta1.png")
p

In [27]:
import plotly.express as plx
p = plx.scatter(y=df["probDelta1"],x=df["demDelta1"], trendline="ols",labels={
    "y": "Default probability yearly change",
    "x": "Democracy index yearly change",
})
p.write_image("probDelta1demDelta1.png")
p

In [28]:
import plotly.express as plx
p=plx.scatter(y=df["probT"],x=df["demT"], trendline="ols",labels={
    "y": "Default probability",
    "x": "Democracy index"})
p.write_image("probTdemT.png")
p

In [14]:
import statsmodels.api as sm
df_ = df[["demDelta1","probDelta1"]].dropna()
df_["+demDelta1"] = df_["demDelta1"]*(df_["demDelta1"]>0)
df_["-demDelta1"] = -df_["demDelta1"]*(df_["demDelta1"]<0)
x = df_[["+demDelta1","-demDelta1"]]
x = sm.add_constant(x)
y = df_["probDelta1"]
r = sm.OLS(y,x).fit().summary()
r

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             probDelta1   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.083
Date:                Sun, 26 Dec 2021   Prob (F-statistic):              0.339
Time:                        01:17:07   Log-Likelihood:                 692.33
No. Observations:                 642   AIC:                            -1379.
Df Residuals:                     639   BIC:                            -1365.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0051      0.004     -1.287      0.199      -0.013       0.003
+demDelta1     0.0146      0.283      0.052      0.959      -0.542       0.571
-demDelta1     0.3665      0.252      1.455      0.146      -0.128       0.861
==============================================================================
Omnibus:                       94.485   Durbin-Watson:                   2.349
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              997.732
Skew:                           0.189   Prob(JB):                    2.21e-217
Kurtosis:                       9.096   Cond. No.                         90.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
import statsmodels.api as sm
df_ = df.dropna()
df_["+demDelta1"] = df_["demDelta1"]*(df_["demDelta1"]>0)
df_["-demDelta1"] = -df_["demDelta1"]*(df_["demDelta1"]<0)
df_["+demDelta2"] = df_["demDelta2"]*(df_["demDelta2"]>0)
df_["-demDelta2"] = -df_["demDelta2"]*(df_["demDelta2"]<0)
df_["+demDelta3"] = df_["demDelta3"]*(df_["demDelta3"]>0)
df_["-demDelta3"] = -df_["demDelta3"]*(df_["demDelta3"]<0)
x = df_[["demT-3","+demDelta1","-demDelta1","+demDelta2","-demDelta2","+demDelta3","-demDelta3",]]
x = sm.add_constant(x)
y = df_["probT"]
sm.OLS(y,x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  probT   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.134
Method:                 Least Squares   F-statistic:                     11.77
Date:                Sun, 26 Dec 2021   Prob (F-statistic):           7.29e-14
Time:                        19:03:06   Log-Likelihood:                 149.55
No. Observations:                 488   AIC:                            -283.1
Df Residuals:                     480   BIC:                            -249.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4860      0.034     14.199      0.000       0.419       0.553
demT-3        -0.3671      0.047     -7.799      0.000      -0.460      -0.275
+demDelta1     0.0206      0.976      0.021      0.983      -1.897       1.938
-demDelta1     1.3624      0.882      1.544      0.123      -0.371       3.096
+demDelta2    -0.6401      1.080     -0.593      0.554      -2.761       1.481
-demDelta2    -0.3768      0.893     -0.422      0.673      -2.132       1.378
+demDelta3     0.5484      0.857      0.640      0.523      -1.136       2.233
-demDelta3     0.9000      0.626      1.437      0.151      -0.330       2.130
==============================================================================
Omnibus:                      146.559   Durbin-Watson:                   0.506
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              367.434
Skew:                           1.509   Prob(JB):                     1.63e-80
Kurtosis:                       5.993   Cond. No.                         203.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""